# Supervised Fine-tuning Gemini 2.5 Flash for Visual Defect Detection

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_visual_defect_detection.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Ftuning%2Fsft_gemini_visual_defect_detection.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/tuning/sft_gemini_visual_defect_detection.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_visual_defect_detection.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_visual_defect_detection.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_visual_defect_detection.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_visual_defect_detection.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_visual_defect_detection.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/sft_gemini_visual_defect_detection.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>


| Author |
| --- |
| [Aniket Agrawal](https://github.com/aniketagrawal2012) |

## Overview

This notebook demonstrates how to perform **supervised fine-tuning** on a Gemini model for a **visual defect detection** task within a manufacturing context. We will use the `google-genai` SDK integrated with Vertex AI to train the model to classify product images and identify flaws.

### Use Case: Classifying Product Quality from Images

We'll fine-tune Gemini to analyze an image of a product from a manufacturing line and classify its quality (e.g., "Pass", "Defect") and provide a short description of the issue if one is found. This is a multimodal task combining image analysis (vision) with text generation (classification and description).

**Workflow:**
1.  **Generate Data**: Create simulated product images (Pass/Defect), upload them to GCS, and create a manifest DataFrame.
2.  **Prepare Tuning Data (JSONL)**: Convert image GCS URIs and corresponding labels (e.g., "Status: Defect - Scratch detected") into the JSON Lines format required for Gemini supervised tuning.
3.  **Upload to GCS**: Store the formatted tuning data (JSONL files) in a Google Cloud Storage bucket.
4.  **Launch Fine-tuning Job**: Use the `google-genai` SDK client (configured for Vertex AI) to start the supervised tuning job.
5.  **Monitor Job**: Track the progress of the fine-tuning job.
6.  **Evaluate Tuned Model**: Make predictions on new product images using the fine-tuned model endpoint and compare qualitatively.
7.  **Integrate Gemini for Reporting**: Use a base Gemini model to summarize the tuning job results.

## Setup

### Install required packages

In [ ]:
import sys  # noqa: F401

# Install necessary libraries
# gcsfs is added to allow pandas to write directly to GCS
# Pillow (PIL) is needed for image generation
!{sys.executable} -m pip install --upgrade --user --quiet pandas numpy google-cloud-aiplatform google-genai google-cloud-storage gcsfs Pillow

**⚠️ Important:** Restart the kernel after installation.

### Authenticate and Initialize Vertex AI

Set your project, region, and GCS bucket information. We configure the notebook for Vertex AI fine-tuning and reporting.

In [ ]:
import subprocess

import vertexai
from google.genai import (
    Client as VertexClient,  # This is for Vertex AI tuning/models client
)

# --- Vertex AI Configuration (Required for Fine-tuning Job) ---
PROJECT_ID = ""  # @param {type: "string", placeholder: "your-gcp-project-id"}
REGION = ""  # @param {type:"string"}
BUCKET_NAME = ""  # @param {type:"string", placeholder: "your-gcs-bucket-name"}
BUCKET_URI = f"gs://{BUCKET_NAME}"


def ensure_gcs_bucket_exists(project_id: str, region: str, bucket_uri: str) -> None:
    """Ensures the specified GCS bucket exists, creating it if necessary."""
    print(f"Checking/Creating bucket: {bucket_uri}")

    # 1. Check if bucket exists. If yes, return immediately (Guard Clause).
    try:
        subprocess.run(
            ["gsutil", "ls", bucket_uri],
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )
        print(f"Bucket {bucket_uri} already exists.")
        return
    except subprocess.CalledProcessError:
        # Fall through only if check failed
        print(f"Bucket {bucket_uri} not found. Attempting to create it.")

    # 2. Create bucket (No longer indented inside an 'else' or 'except' block)
    try:
        subprocess.run(
            ["gsutil", "mb", "-l", region, "-p", project_id, bucket_uri],
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )
        print(f"Bucket {bucket_uri} created successfully.")
    except subprocess.CalledProcessError as e:
        error_msg = (
            f"Failed to create bucket {bucket_uri}. Error: {e.stderr.decode().strip()}"
        )
        raise ValueError(error_msg) from e


# --- Authentication (Colab/Workbench for Vertex AI) ---
if not PROJECT_ID or PROJECT_ID == "":
    try:
        from google.colab import auth

        auth.authenticate_user()

        PROJECT_ID = (
            subprocess.check_output(["gcloud", "config", "get-value", "project"])
            .decode("utf-8")
            .strip()
        )
        print(f"Retrieved Project ID: {PROJECT_ID}")
    except Exception as e:
        print(
            f"Could not automatically retrieve Project ID. Please set it manually. Error: {e}"
        )

# Ensure BUCKET_NAME is set, and attempt to create the bucket
if not BUCKET_NAME or BUCKET_NAME == "":
    if PROJECT_ID:
        BUCKET_NAME = f"{PROJECT_ID}-gemini-tuning-bucket"
        BUCKET_URI = f"gs://{BUCKET_NAME}"
        print(f"Bucket name not provided. Using default: {BUCKET_NAME}")
    else:
        raise ValueError(
            "Please provide a valid GCS Bucket name or ensure PROJECT_ID is set for default bucket creation."
        )

ensure_gcs_bucket_exists(PROJECT_ID, REGION, BUCKET_URI)

if PROJECT_ID:
    print(
        f"Initializing Vertex AI for project: {PROJECT_ID} in {REGION} using bucket {BUCKET_URI}"
    )
    # Initialize Vertex AI SDK (needed for launching the tuning job)
    vertexai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)
    # Initialize the genai client specifically for Vertex AI operations (like tuning)
    vertex_client = VertexClient(vertexai=True, project=PROJECT_ID, location=REGION)
    print("Vertex AI SDK Initialized.")
else:
    raise ValueError("PROJECT_ID must be set for Vertex AI operations.")

### Imports and Global Configuration

In [ ]:
import io
import json
import random
import time
import warnings
from typing import Any

import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
from google.cloud import storage
from google.genai import types as genai_types

# --- Global Settings ---
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
np.random.seed(42)
random.seed(42)

# --- Constants ---
BASE_MODEL_ID = "gemini-2.5-flash"  # Tunable model ID on Vertex AI
TUNED_MODEL_DISPLAY_NAME = f"visual-defect-gemini-tuned-{int(time.time())}"
DATA_DIR_GCS = f"{BUCKET_URI}/visual_defect_tuning_data"
IMAGE_DIR_GCS_PATH = "visual_defect_tuning_data/images"  # Relative path for client
IMAGE_DIR_GCS_URI = f"{DATA_DIR_GCS}/images"
TRAIN_JSONL_GCS_URI = f"{DATA_DIR_GCS}/train_data.jsonl"
VALIDATION_JSONL_GCS_URI = f"{DATA_DIR_GCS}/validation_data.jsonl"
TEST_JSONL_GCS_URI = f"{DATA_DIR_GCS}/test_data.jsonl"  # For qualitative eval later

print(f"Base model for tuning: {BASE_MODEL_ID}")
print(f"Tuning data GCS path: {DATA_DIR_GCS}")
print(f"Image GCS URI: {IMAGE_DIR_GCS_URI}")

## Step 1: Generate Simulated Image Data

Instead of loading data, we'll generate simulated product images (simple shapes) and upload them to GCS. We'll create 'Pass' images (clean) and 'Defect' images (with a visual flaw). We will return a Pandas DataFrame acting as a manifest file.

In [ ]:
def _upload_image_blob(bucket, blob_name: str, image: Image.Image) -> bool:
    """Helper to safely upload a PIL image to GCS."""
    try:
        blob = bucket.blob(blob_name)
        with io.BytesIO() as output:
            image.save(output, format="PNG")
            blob.upload_from_string(output.getvalue(), content_type="image/png")
        return True
    except Exception as e:
        print(f"Failed to upload {blob_name}: {e}")
        return False


def generate_and_upload_images(
    bucket_name: str,
    gcs_image_path: str,
    num_images: int = 100,
    defect_ratio: float = 0.4,
) -> pd.DataFrame:
    """Generates simple images, uploads to GCS, and returns a manifest DataFrame."""
    print(f"Generating {num_images} simulated images...")
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.bucket(bucket_name)
    manifest = []

    for i in range(num_images):
        img = Image.new("RGB", (100, 100), color="#DDDDDD")
        draw = ImageDraw.Draw(img)
        draw.rectangle((20, 20, 80, 80), fill="#5555AA")  # Main product shape

        is_defect = random.random() < defect_ratio
        image_name = f"product_image_{i}.png"
        gcs_blob_name = f"{gcs_image_path}/{image_name}"
        gcs_uri = f"gs://{bucket_name}/{gcs_blob_name}"
        defect_type = "None"
        label = "Status: Pass"

        if is_defect:
            # Add a random defect
            defect_type = random.choice(["Scratch", "Crack", "Discoloration"])
            if defect_type == "Scratch":
                draw.line((30, 30, 70, 70), fill="#FF3333", width=2)
                label = "Status: Defect - Scratch detected on product surface."
            elif defect_type == "Crack":
                draw.line((30, 50, 70, 45), fill="#FFFFFF", width=3)
                label = "Status: Defect - Crack identified in main body."
            elif defect_type == "Discoloration":
                draw.ellipse((55, 55, 75, 75), fill="#44AA44")
                label = "Status: Defect - Discoloration spot found."

        # Upload to GCS
        if not _upload_image_blob(bucket, gcs_blob_name, img):
            continue

        manifest.append(
            {
                "image_name": image_name,
                "gcs_uri": gcs_uri,
                "status": "Defect" if is_defect else "Pass",
                "defect_type": defect_type,
                "label": label,
            }
        )

        if (i + 1) % 20 == 0:
            print(f"  ...generated and uploaded {i + 1}/{num_images} images.")

    print(f"Image generation complete. Uploaded {len(manifest)} images.")
    return pd.DataFrame(manifest)


# Generate data (e.g., 200 samples for this demo)
# For a real project, you'd need many more (100s or 1000s)
image_manifest_df = generate_and_upload_images(
    bucket_name=BUCKET_NAME,
    gcs_image_path=IMAGE_DIR_GCS_PATH,
    num_images=200,
    defect_ratio=0.5,
)

print("\n--- Image Manifest Sample ---")
print(image_manifest_df.head())

## Step 2: Prepare Tuning Data (JSONL)

We convert the manifest DataFrame into the required JSON Lines format. Each line will contain a **multimodal prompt** (text + image) and the expected completion (the classification label).

In [ ]:
def _build_tuning_example(prompt: str, image_uri: str, label: str) -> dict:
    """Constructs the dictionary for a single tuning example."""
    return {
        "contents": [
            {
                "role": "user",
                "parts": [
                    {"text": prompt},
                    {"fileData": {"mimeType": "image/png", "fileUri": image_uri}},
                ],
            },
            {"role": "model", "parts": [{"text": label}]},
        ]
    }


def create_tuning_jsonl_from_manifest(
    manifest_df: pd.DataFrame,
) -> list[dict[str, Any]]:
    """Creates JSONL data for Gemini multimodal supervised tuning."""
    print("\n--- Preparing JSONL Tuning Data ---")

    jsonl_data = []
    base_prompt = "Analyze the following product image for manufacturing defects. Classify its status as 'Pass' or 'Defect' and provide a brief description if a defect is present."

    for _, row in manifest_df.iterrows():
        image_uri = row["gcs_uri"]
        target_label = row["label"]

        if not image_uri or pd.isna(image_uri):
            print("Skipping row with missing image URI.")
            continue

        # Format according to Gemini multimodal tuning requirements
        # The 'user' role contains both the text prompt and the image file data.
        instance = _build_tuning_example(base_prompt, image_uri, target_label)
        jsonl_data.append(instance)

    print(f"Generated {len(jsonl_data)} JSONL instances.")
    return jsonl_data


# Create JSONL data
tuning_data_jsonl = create_tuning_jsonl_from_manifest(image_manifest_df)

# Shuffle and Split data
if tuning_data_jsonl:
    random.shuffle(tuning_data_jsonl)
    # Using 80% train, 10% validation, 10% test split
    split_idx_val = int(len(tuning_data_jsonl) * 0.8)
    split_idx_test = int(len(tuning_data_jsonl) * 0.9)

    train_split = tuning_data_jsonl[:split_idx_val]
    validation_split = tuning_data_jsonl[split_idx_val:split_idx_test]
    test_split = tuning_data_jsonl[split_idx_test:]

    print(
        f"Split sizes: Train={len(train_split)}, Validation={len(validation_split)}, Test={len(test_split)}"
    )

    # Display a sample
    print("\n--- Sample JSONL Instance ---")
    print(json.dumps(train_split[0], indent=2))
else:
    print("\nWarning: No tuning data generated.")
    # Initialize splits as empty lists to prevent errors later
    train_split, validation_split, test_split = [], [], []

## Step 3: Upload Tuning Data to GCS

The fine-tuning service reads data directly from Google Cloud Storage.

In [ ]:
import google.auth

def save_jsonl_to_gcs(instances: list[dict[str, Any]], gcs_uri: str) -> None:
    """Saves a list of dictionaries as a JSONL file to GCS using Pandas."""
    if not instances:
        print(f"No instances to upload to {gcs_uri}. Skipping upload.")
        return

    print(f"Uploading {len(instances)} instances to {gcs_uri}...")

    try:
        # Get the application default credentials
        credentials, _ = google.auth.default()

        # Convert list of dicts to DataFrame
        df = pd.DataFrame(instances)

        # Save DataFrame to GCS as JSONL
        # We MUST pass the 'token' (credentials) to authenticate the request
        storage_options = {"project": PROJECT_ID, "token": credentials}

        df.to_json(
            gcs_uri, orient="records", lines=True, storage_options=storage_options
        )

        print("Upload complete.")
    except Exception as e:
        print(f"ERROR during GCS upload to {gcs_uri}: {e}")
        print(
            "Please ensure your GCS bucket is accessible and pandas has GCS permissions (installed via gcsfs)."
        )


# Save splits to GCS
save_jsonl_to_gcs(train_split, TRAIN_JSONL_GCS_URI)
save_jsonl_to_gcs(validation_split, VALIDATION_JSONL_GCS_URI)
save_jsonl_to_gcs(
    test_split, TEST_JSONL_GCS_URI
)  # Save test split for later evaluation

## Step 4: Launch Fine-tuning Job

We use the `google-genai` client **configured for Vertex AI** (`vertex_client`) to start the supervised tuning job, as fine-tuning management is a Vertex AI feature.

In [ ]:
TUNING_JOB_NAME = None  # Initialize
if not train_split or not validation_split:
    print("Skipping fine-tuning job launch as training or validation data is empty.")
else:
    print(f"Starting supervised fine-tuning job for model: {BASE_MODEL_ID}")
    print(f"Tuned model display name: {TUNED_MODEL_DISPLAY_NAME}")

    training_dataset = {
        "gcs_uri": TRAIN_JSONL_GCS_URI,
    }

    validation_dataset = genai_types.TuningValidationDataset(
        gcs_uri=VALIDATION_JSONL_GCS_URI
    )

    try:
        # Use the vertex_client configured specifically for Vertex AI operations
        sft_tuning_job = vertex_client.tunings.tune(
            base_model=BASE_MODEL_ID,
            training_dataset=training_dataset,
            config=genai_types.CreateTuningJobConfig(
                adapter_size="ADAPTER_SIZE_FOUR",  # Smaller adapter for faster tuning
                epoch_count=5,  # Increased epochs for better specialization
                tuned_model_display_name=TUNED_MODEL_DISPLAY_NAME,
                validation_dataset=validation_dataset,
            ),
        )
        print("\nTuning job created:")
        print(sft_tuning_job)
        TUNING_JOB_NAME = sft_tuning_job.name  # Save for monitoring

    except Exception as e:
        print(f"ERROR starting tuning job: {e}")
        # Attempt to list existing jobs with the same display name in case of interruption
        try:
            print(
                f"Checking for existing tuning jobs named '{TUNED_MODEL_DISPLAY_NAME}'..."
            )
            existing_jobs = vertex_client.tunings.list(
                page_size=100
            )  # List might need pagination for many jobs
            for job in existing_jobs:
                # Check if config exists and has the attribute
                job_config = getattr(job, "config", None)
                if (
                    job_config
                    and getattr(job_config, "tuned_model_display_name", None)
                    == TUNED_MODEL_DISPLAY_NAME
                ):
                    print(f"Found existing job: {job.name} with state {job.state}")
                    TUNING_JOB_NAME = job.name  # Use the existing job name
                    break
        except Exception as list_e:
            print(f"Could not list existing tuning jobs: {list_e}")

**Note:** Fine-tuning can take a significant amount of time (potentially 30 minutes to several hours depending on the dataset size, base model, and adapter size).

## Step 5: Monitor Job

In [ ]:
def monitor_tuning_job(job_name: str) -> Any:
    """Polls the tuning job until it reaches a terminal state."""
    print(f"Monitoring tuning job: {job_name}")
    running_states = {
        genai_types.JobState.JOB_STATE_PENDING,
        genai_types.JobState.JOB_STATE_RUNNING,
    }

    while True:
        try:
            tuning_job = vertex_client.tunings.get(name=job_name)
        except Exception as e:
            print(f"Error polling status: {e}. Retrying in 60s...")
            time.sleep(60)
            continue

        if tuning_job.state not in running_states:
            return tuning_job

        state_name = str(tuning_job.state).split(".")[-1]
        print(f"  Current state: {state_name}...")
        time.sleep(60)


TUNED_MODEL_ENDPOINT = None

if TUNING_JOB_NAME:
    final_job = monitor_tuning_job(TUNING_JOB_NAME)

    final_state_name = str(final_job.state).split(".")[-1]
    print(f"\nTuning job finished with state: {final_state_name}")

    if final_job.state == genai_types.JobState.JOB_STATE_SUCCEEDED:
        if hasattr(final_job, "tuned_model") and final_job.tuned_model.endpoint:
            TUNED_MODEL_ENDPOINT = final_job.tuned_model.endpoint
            print(f"Tuned model endpoint ready: {TUNED_MODEL_ENDPOINT}")
        else:
            print("Warning: Job succeeded but endpoint not found.")
    else:
        error_msg = getattr(final_job, "error", "Unknown error")
        print(f"Tuning job failed. Error: {error_msg}")
else:
    print("Skipping monitoring...")

## Step 6: Evaluate Tuned Model (Qualitative)

We take a sample from our test set (which the model hasn't seen during tuning) and send the multimodal prompt (text + image) to the tuned endpoint to compare the prediction to the expected output.

In [ ]:
import re  # <-- Import regex module

from google.genai import types as genai_types  # Ensure genai_types is available

def _extract_predicted_text(response: Any) -> str:
    """Safely extracts the predicted text using guard clauses."""
    if not response:
        return "(Response is None)"

    # 1. Check simplified text attribute
    if hasattr(response, "text") and response.text:
        return response.text.strip()

    # 2. Validate candidates exist
    if not hasattr(response, "candidates") or not response.candidates:
        return "(No candidates found)"

    first_candidate = response.candidates[0]

    # 3. Validate finish reason
    finish_reason = getattr(first_candidate, "finish_reason", None)
    if finish_reason != genai_types.FinishReason.STOP:
        return f"(Generation stopped: {finish_reason})"

    # 4. Validate content parts
    if not (hasattr(first_candidate, "content") and first_candidate.content.parts):
        return "(No content parts)"

    return first_candidate.content.parts[0].text.strip()


def evaluate_qualitatively(
    tuned_endpoint: str, test_data: list[dict[str, Any]], num_samples: int = 3
) -> None:
    """Makes predictions with the tuned model and prints comparisons."""
    if not tuned_endpoint:
        print("Tuned model endpoint not available. Skipping evaluation.")
        return

    if not test_data:
        print("No test data available for evaluation.")
        return

    print(f"\n--- Qualitative Evaluation of Tuned Model ({tuned_endpoint}) ---")

    # Select random samples from the test set
    samples = random.sample(test_data, min(num_samples, len(test_data)))

    for i, sample in enumerate(samples):
        print(f"\n--- Sample {i + 1} ---")
        # Ensure the sample structure is correct
        try:
            # Extract multimodal prompt parts
            user_parts = sample["contents"][0]["parts"]
            expected_output = sample["contents"][1]["parts"][0]["text"]

            # Reconstruct the text and image parts for the prompt
            prompt_text_part = user_parts[0]["text"]
            image_file_part = user_parts[1]["fileData"]
            image_uri = image_file_part["fileUri"]
            image_mime = image_file_part["mimeType"]

        except (KeyError, IndexError, TypeError) as e:
            print(f"Skipping sample due to unexpected format: {e}")
            print(f"Problematic sample: {sample}")
            continue

        print(f"Input Prompt Text: {prompt_text_part}")
        print(f"Input Image URI: {image_uri}")
        print(f"\nExpected Output: {expected_output}")

        try:
            # Prepare contents for prediction (both text and image parts)
            prediction_contents = [
                {
                    "role": "user",
                    "parts": [
                        {"text": prompt_text_part},
                        {"fileData": {"mimeType": image_mime, "fileUri": image_uri}},
                    ],
                }
            ]

            # Use the vertex_client for predictions against the tuned endpoint
            response = vertex_client.models.generate_content(
                model=tuned_endpoint,
                contents=prediction_contents,
                config={
                    "temperature": 0.1,  # Low temperature for deterministic classification
                    "max_output_tokens": 2000,
                },
            )

            # Safely access predicted text
            predicted_output = _extract_predicted_text(response)

            print(f"Predicted Output: {predicted_output}")

            # --- NEW REGEX EVALUATION LOGIC ---
            result_str = "MISMATCH"  # Default

            if expected_output == "Status: Pass":
                if predicted_output == "Status: Pass":
                    result_str = "MATCH"
            elif "Defect" in expected_output:
                # 1. Check if prediction also says "Defect"
                if "Defect" not in predicted_output:
                    result_str = "MISMATCH (Predicted 'Pass' for 'Defect')"
                else:
                    # 2. Check if the key defect word is present
                    key_defect_match = re.search(
                        r"(Scratch|Crack|Discoloration)", expected_output
                    )
                    if key_defect_match:
                        defect_type = key_defect_match.group(1)
                        # Check if the predicted string contains the defect type (case-insensitive)
                        if re.search(defect_type, predicted_output, re.IGNORECASE):
                            result_str = "MATCH (Regex)"
                        else:
                            result_str = f"MISMATCH (Missing key defect: {defect_type})"
                    else:
                        # Fallback if it's a defect but not one of the known types
                        result_str = "MATCH (Regex - 'Defect' present)"

            print(f"Result: {result_str}")
            # --- END OF NEW LOGIC ---

        except Exception as e:
            print(f"ERROR during prediction for sample {i + 1}: {e}")


# Run qualitative evaluation (only if tuning succeeded and test data exists)
evaluate_qualitatively(TUNED_MODEL_ENDPOINT, test_split)

## Step 7: Integrating Gemini for Reporting (Using Base Model)

We can use a base Gemini model (accessed via Vertex AI) to summarize the fine-tuning job itself.

In [ ]:
def generate_tuning_summary_with_gemini(tuning_job_details: Any) -> None:
    """Generates a summary of the tuning job using the Gemini API."""
    print("\n--- Generating Tuning Job Summary with Gemini ---")

    if not tuning_job_details:
        print("No tuning job details provided. Skipping summary.")
        return

    # We will use the Vertex AI client, which is already initialized.
    model_name_for_vertex_ai = "gemini-2.5-flash"  # Use a standard model for reporting
    reporting_client = None

    try:
        # This uses the high-level vertexai SDK for base model generation
        # Correctly import GenerativeModel from vertexai.preview.generative_models
        from vertexai.preview.generative_models import GenerativeModel

        reporting_client = GenerativeModel(model_name_for_vertex_ai)
        print(f"Using Vertex AI model ({model_name_for_vertex_ai}) for reporting.")
    except Exception as e:
        print(
            f"Failed to initialize Vertex AI client for reporting with {model_name_for_vertex_ai}: {e}"
        )
        print("Skipping summary generation.")
        return

    try:
        # Extract relevant details safely
        job_name = getattr(tuning_job_details, "name", "N/A")
        job_state_enum = getattr(
            tuning_job_details, "state", genai_types.JobState.JOB_STATE_UNSPECIFIED
        )  # Default to unspecified
        job_state = str(job_state_enum).split(".")[
            -1
        ]  # Get 'SUCCEEDED', 'FAILED', etc.
        base_model = getattr(tuning_job_details, "base_model", "N/A")
        tuned_model_obj = getattr(tuning_job_details, "tuned_model", None)
        tuned_endpoint = (
            getattr(tuned_model_obj, "endpoint", "N/A") if tuned_model_obj else "N/A"
        )
        error_obj = getattr(tuning_job_details, "error", None)
        error_message = str(error_obj) if error_obj else "None"
        config_obj = getattr(tuning_job_details, "config", None)
        display_name = (
            getattr(config_obj, "tuned_model_display_name", "N/A")
            if config_obj
            else "N/A"
        )

        prompt = f"""Generate a brief status report for a Gemini model fine-tuning job for a 'Visual Defect Detection' use case.
        Job Name: {job_name}
        Base Model: {base_model}
        Tuned Model Display Name: {display_name}
        Final Status: {job_state}
        Tuned Model Endpoint: {tuned_endpoint}
        Error (if any): {error_message}

        Summarize the outcome of this tuning job in 1-2 sentences, specifically mentioning its readiness for the manufacturing defect analysis task."""

        print("\nSending request to Gemini...")
        # Use the selected reporting_client (Vertex AI based)
        response = reporting_client.generate_content(prompt)

        print("\n--- Gemini Tuning Job Summary ---")
        # Handle potential response variations
        response_text = _extract_predicted_text(response)
        print(response_text)
        print("---------------------------------")

    except Exception as e:
        print(f"\nERROR: Failed to generate Gemini summary: {e}")
        if (
            "permission denied" in str(e).lower()
            or "consumer project" in str(e).lower()
        ):
            print(
                "Please ensure the Vertex AI API is enabled in your project and the runtime environment has the correct permissions."
            )
        else:
            print(
                "Please check your Vertex AI setup, model name, and network connection."
            )


# Get the final job details again using the vertex_client (which manages tuning)
final_tuning_job = None
if TUNING_JOB_NAME:
    try:
        # Use vertex_client to get the job status
        final_tuning_job = vertex_client.tunings.get(name=TUNING_JOB_NAME)
    except Exception as e:
        print(f"Error retrieving final tuning job details: {e}")

# Generate the summary using the Vertex Gemini client
generate_tuning_summary_with_gemini(final_tuning_job)